# import

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import glob
import datetime as dt
import tqdm
import json
import matplotlib.pyplot as plt
import numpy as np
import sys, os

sys.path.insert(0, os.path.abspath('../src/'))
from model.vgg19 import VGG19
from model.ViT import VisionTransformer
from model.augmentation import aug_process
from dataloader import dataloader

# model type

In [2]:
model_type = 'EfficientNet'

# file path

In [3]:
json_open = open('../config.json', 'r')
config = json.load(json_open)

In [4]:
dataset_path = f'{config["filepath"]["dataset"]}'
partitions_path = f'{config["filepath"]["partitions"]}'
output_path = f'{config["filepath"]["output"]}'
file_name = f"{model_type}_{config['data']['img_size']}_keras_baseline"
log_dir = f"{output_path}/logs/{file_name}"

os.makedirs(log_dir, exist_ok=True)

In [5]:
test_txt_path_list = glob.glob(f"{partitions_path}/Test*")
train_txt_path_list = glob.glob(f"{partitions_path}/Train*")
class_name_path = f"{partitions_path}/ClassName.txt"

# label to index

In [6]:
f = open(class_name_path, 'r')
label_name_list = f.readlines()
label_name_list = list(map(lambda tmp_path: tmp_path[:-1].split('/', 2)[2], label_name_list))
f.close()
label_to_index = dict((name, index) for index, name in enumerate(label_name_list))

# text to path list

In [7]:
def txt_to_path(txt_path_list):
    path_list = []
    for path in txt_path_list:
        f = open(path, 'r')
        path_list+=f.readlines()
        f.close()
    
    path_list = list(map(lambda tmp_path: dataset_path+tmp_path[:-1], path_list))#.remove(config["data"]["exclude_list"])
    path_list = sorted(list(set(path_list)-set(config["data"]["exclude_list"])))
    label_list = list(map(lambda tmp_path: label_to_index[tmp_path.split('/', 6)[6].rsplit('/', 1)[0]], path_list))
    return path_list, label_list

In [8]:
img_path_list, label_list = txt_to_path(train_txt_path_list)

In [9]:
train_img_path_list, val_img_path_list,\
train_label_list, val_label_list = train_test_split(img_path_list, label_list,
                                                    test_size=0.2, random_state=0)
test_img_path_list, test_label_list = txt_to_path(test_txt_path_list)

In [10]:
# cannot use same batch size as mixed float16
tmp_dataloader = dataloader(config['data']['batch_size']//2, 
                            config['data']['img_size'],
                            is_aug=True)

train_ds = tmp_dataloader(train_img_path_list, train_label_list, shuffle_buffer=100)
val_ds = tmp_dataloader(val_img_path_list, val_label_list, is_train=False, shuffle=False)

# def model

In [11]:
strategy = tf.distribute.MirroredStrategy()
num_classes = len(label_to_index)+1
with strategy.scope():
    if model_type=='vgg19':
        model = VGG19(num_classes, img_size=config['data']['img_size'],)
    if model_type=='ViT':
        model = VisionTransformer(num_classes=num_classes, img_size=config['data']['img_size'])
    if model_type=='EfficientNet':
        model = tf.keras.applications.EfficientNetB3(classes=num_classes, weights=None,
                                                     input_shape=(config['data']['img_size'],
                                                                  config['data']['img_size'], 3))
    
optimizer = tfa.optimizers.RectifiedAdam(lr=config['data']['lr']/2, clipnorm=0.01)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy', metrics=['acc'],)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

# train

In [12]:
%%time
TB = tf.keras.callbacks.TensorBoard(log_dir=f'{output_path}/logs/{file_name}',)
RLRP = tf.keras.callbacks.ReduceLROnPlateau(factor=0.95, patience=2)
hist = model.fit(train_ds, validation_data=val_ds,
                 epochs=config['data']['epochs'], callbacks=[TB, RLRP])

Epoch 1/8
INFO:tensorflow:batch_all_reduce: 340 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 340 all-reduces with algorithm = nccl, num_packs = 1
2004/2004 [==============================] - 1118s 482ms/step - loss: 5.9815 - acc: 0.0053 - val_loss: 5.7409 - val_acc: 0.0158
Epoch 2/8
2004/2004 [==============================] - 955s 476ms/step - loss: 5.6326 - acc: 0.0151 - val_loss: 5.1789 - val_acc: 0.0339
Epoch 3/8
2004/2004 [==============================] - 955s 476ms/step - loss: 5.3086 - acc: 0.0284 - val_loss: 4.9416 - val_acc: 0.0474
Epoch 4/8
2004/2004 [==============================] - 954s 476ms/step - loss: 5.0700 - acc: 0.0405 - val_loss: 4.7804 - val_acc: 0.0607
Epoch 5/8
2004/2004 [==============================] - 951s 474ms/step - loss: 4.8438 - acc: 0.0563 - val_loss: 4.5499 - val_acc: 0.0839
Epoch 6/8
2004/2004 [==============================] - 953s 475ms/step - loss: 4.6751 - acc: 0.0710 - val_loss: 4.3873 - val_acc: 0.0984
Epo